# 株価取得処理

In [1]:
# ライブラリのインストール
!pip install yfinance


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import yfinance as yf
import pandas as pd

# Pandasの表示オプションを設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# トヨタ自動車 (7203.T) のデータを取得
ticker_code = '7203.T'
ticker = yf.Ticker(ticker_code)

# 1. 株価データの取得 (過去1年分の日足)
print("--- 1. 株価データ (過去5日分) ---")
hist = ticker.history(period="1y")
print(hist.tail()) # .tail()で末尾5日分を表示

print("\n" + "="*50 + "\n")

# 2. 企業情報・指標の取得
print("--- 2. 企業情報・指標 ---")
info = ticker.info

# 必要な情報を抽出して表示
print(f"企業名: {info.get('longName')}")
print(f"業種: {info.get('industry')}")
print(f"株価: {info.get('currentPrice')}")
print(f"PER (株価収益率): {info.get('trailingPE')}")
print(f"PBR (株価純資産倍率): {info.get('priceToBook')}")
print(f"配当利回り: {info.get('dividendYield')}")

# 3. 配当性向の取得
# infoの中に'payoutRatio'というキーで配当性向が含まれている
print("\n--- 3. 配当性向 ---")
payout_ratio = info.get('payoutRatio')
if payout_ratio:
    print(f"配当性向: {payout_ratio:.2%}") # パーセント表示
else:
    print("配当性向データがありません。")

--- 1. 株価データ (過去5日分) ---
                             Open    High     Low   Close    Volume  Dividends  Stock Splits
Date                                                                                        
2025-06-11 00:00:00+09:00  2670.0  2678.0  2652.0  2656.0  16367500        0.0           0.0
2025-06-12 00:00:00+09:00  2645.0  2648.0  2608.0  2616.5  22784400        0.0           0.0
2025-06-13 00:00:00+09:00  2586.0  2589.0  2539.0  2555.0  36054800        0.0           0.0
2025-06-16 00:00:00+09:00  2580.5  2586.0  2554.0  2554.5  19281900        0.0           0.0
2025-06-17 00:00:00+09:00  2535.0  2551.0  2530.5  2538.0  18263000        0.0           0.0


--- 2. 企業情報・指標 ---
企業名: Toyota Motor Corporation
業種: Auto Manufacturers
株価: 2538.0
PER (株価収益率): 7.056468
PBR (株価純資産倍率): 0.9218746
配当利回り: 3.72

--- 3. 配当性向 ---
配当性向: 25.03%


In [15]:
div_yield = info.get('dividendYield')


print(f"配当利回り: {div_yield}")
print(f"配当利回り: {div_yield:.2f}%")





配当利回り: 3.72
配当利回り: 3.72%


In [17]:
import yfinance as yf
import pandas as pd
import numpy as np # RCIの計算に必要

# --- 関数定義 ---

def add_rci(df, period):
    """
    DataFrameにRCI（順位相関指数）の列を追加する関数
    
    Args:
        df (pd.DataFrame): 'Close'列を含む株価データ
        period (int): RCIの計算期間
        
    Returns:
        pd.DataFrame: RCI列が追加されたDataFrame
    """
    # RCIを計算するための内部関数
    def rci_calc(series):
        n = len(series)
        # 日付の順位は 1, 2, ..., n
        date_rank = pd.Series(range(1, n + 1))
        # 価格の順位
        price_rank = pd.Series(series).rank(method='first')
        # d (順位の差の二乗和) を計算
        d = ((date_rank - price_rank.reset_index(drop=True)) ** 2).sum()
        # RCIを計算
        return (1 - (6 * d) / (n * (n**2 - 1))) * 100

    column_name = f'RCI_{period}'
    # rolling().apply() を使って各期間でRCIを計算
    df[column_name] = df['Close'].rolling(window=period).apply(rci_calc, raw=True)
    return df

def calculate_fibonacci(df):
    """
    フィボナッチ・リトレースメントのレベルを計算する関数
    
    Args:
        df (pd.DataFrame): 'High'と'Low'列を含む株価データ
        
    Returns:
        tuple: (フィボナッチレベルの辞書, 期間最高値, 期間最安値)
    """
    # 期間内の最高値と最安値を取得
    high_price = df['High'].max()
    low_price = df['Low'].min()
    
    # 値幅
    diff = high_price - low_price
    
    # フィボナッチ比率
    ratios = [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
    
    levels = {}
    print(f"フィボナッチ・リトレースメント (押し目・サポートの目安)")
    print(f"期間最高値: {high_price:,.2f} 円")
    print(f"期間最安値: {low_price:,.2f} 円")
    print("-" * 30)

    # 各レベルの価格を計算（高値からの下落率で計算）
    for ratio in sorted(ratios, reverse=True):
        level_price = high_price - diff * ratio
        levels[f'{ratio*100:.1f}%'] = level_price
        
    return levels

# --- メイン処理 ---

# Pandasの表示オプションを設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# 銘柄コードとデータ取得期間
ticker_code = '4751.T' # 銘柄を指定
period_to_get = "1y"   # 過去1年分

# Tickerオブジェクトを作成
ticker = yf.Ticker(ticker_code)

# 1. 企業情報の取得
print("--- 1. 企業情報・指標 ---")
info = ticker.info
print(f"企業名: {info.get('longName')}")
print(f"業種: {info.get('industry')}")
print(f"株価: {info.get('currentPrice', 'N/A'):,.2f} 円")
print(f"PER (株価収益率): {info.get('trailingPE', 'N/A'):.2f} 倍")
print(f"PBR (株価純資産倍率): {info.get('priceToBook', 'N/A'):.2f} 倍")

# 配当利回りはパーセント表示に修正
div_yield = info.get('dividendYield')
if div_yield:
    print(f"配当利回り: {div_yield:.2f}%")
else:
    print(f"配当利回り: N/A")

print("\n" + "="*50 + "\n")


# 2. 株価データとRCIの取得・計算
print(f"--- 2. 最新の株価とRCI (対象期間: {period_to_get}) ---")
hist = ticker.history(period=period_to_get)

# 短期・中期・長期のRCIを計算してDataFrameに追加
hist = add_rci(hist, 9)
hist = add_rci(hist, 26)
hist = add_rci(hist, 52)

# 最新5日間の終値とRCIを表示
print(hist[['Close', 'RCI_9', 'RCI_26', 'RCI_52']].tail())

print("\n" + "="*50 + "\n")


# 3. フィボナッチ・リトレースメントの計算と表示
print(f"--- 3. フィボナッチライン (対象期間: {period_to_get}) ---")
fib_levels = calculate_fibonacci(hist)

# 計算結果を整形して表示
for level_name, level_price in fib_levels.items():
    print(f"  {level_name.rjust(6)}: {level_price:,.2f} 円")

print("\n" + "="*50 + "\n")


# 4. 配当性向の取得
print("--- 4. 配当性向 ---")
payout_ratio = info.get('payoutRatio')
if payout_ratio:
    # 0~1の範囲で返ってくるのでパーセント表示
    print(f"配当性向: {payout_ratio:.2%}")
else:
    print("配当性向データがありません。")

--- 1. 企業情報・指標 ---
企業名: CyberAgent, Inc.
業種: Advertising Agencies
株価: 1,509.50 円
PER (株価収益率): 35.49 倍
PBR (株価純資産倍率): 4.59 倍
配当利回り: 2.19%


--- 2. 最新の株価とRCI (対象期間: 1y) ---
                            Close      RCI_9     RCI_26     RCI_52
Date                                                              
2025-06-11 00:00:00+09:00  1500.0  33.333333  91.452991  90.233074
2025-06-12 00:00:00+09:00  1485.5 -13.333333  90.564103  94.262785
2025-06-13 00:00:00+09:00  1465.5 -73.333333  85.641026  95.031162
2025-06-16 00:00:00+09:00  1508.0 -43.333333  84.888889  95.466576
2025-06-17 00:00:00+09:00  1509.5   8.333333  84.341880  95.910527


--- 3. フィボナッチライン (対象期間: 1y) ---
フィボナッチ・リトレースメント (押し目・サポートの目安)
期間最高値: 1,538.50 円
期間最安値: 785.21 円
------------------------------
  100.0%: 785.21 円
   78.6%: 946.41 円
   61.8%: 1,072.97 円
   50.0%: 1,161.85 円
   38.2%: 1,250.74 円
   23.6%: 1,360.72 円
    0.0%: 1,538.50 円


--- 4. 配当性向 ---
配当性向: 37.66%


In [5]:
pip install --upgrade yfinance



[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
import yfinance as yf
import pandas as pd
import numpy as np # RCIの計算に必要

# --- 関数定義 ---

def add_rci(df, period):
    """
    DataFrameにRCI（順位相関指数）の列を追加する関数
    
    Args:
        df (pd.DataFrame): 'Close'列を含む株価データ
        period (int): RCIの計算期間
        
    Returns:
        pd.DataFrame: RCI列が追加されたDataFrame
    """
    # RCIを計算するための内部関数
    def rci_calc(series):
        n = len(series)
        # 日付の順位は 1, 2, ..., n
        date_rank = pd.Series(range(1, n + 1))
        # 価格の順位
        price_rank = pd.Series(series).rank(method='first')
        # d (順位の差の二乗和) を計算
        d = ((date_rank - price_rank.reset_index(drop=True)) ** 2).sum()
        # RCIを計算
        return (1 - (6 * d) / (n * (n**2 - 1))) * 100

    column_name = f'RCI_{period}'
    # rolling().apply() を使って各期間でRCIを計算
    df[column_name] = df['Close'].rolling(window=period).apply(rci_calc, raw=True)
    return df

def calculate_fibonacci(df):
    """
    フィボナッチ・リトレースメントのレベルを計算する関数
    
    Args:
        df (pd.DataFrame): 'High'と'Low'列を含む株価データ
        
    Returns:
        tuple: (フィボナッチレベルの辞書, 期間最高値, 期間最安値)
    """
    # 期間内の最高値と最安値を取得
    high_price = df['High'].max()
    low_price = df['Low'].min()
    
    # 値幅
    diff = high_price - low_price
    
    # フィボナッチ比率
    ratios = [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
    
    levels = {}
    print(f"フィボナッチ・リトレースメント (押し目・サポートの目安)")
    print(f"期間最高値: {high_price:,.2f} 円")
    print(f"期間最安値: {low_price:,.2f} 円")
    print("-" * 30)

    # 各レベルの価格を計算（高値からの下落率で計算）
    for ratio in sorted(ratios, reverse=True):
        level_price = high_price - diff * ratio
        levels[f'{ratio*100:.1f}%'] = level_price
        
    return levels

# --- メイン処理 ---

# Pandasの表示オプションを設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# 銘柄コードとデータ取得期間
ticker_code = '4751.T' # トヨタ自動車
period_to_get = "1y"   # 過去1年分

# Tickerオブジェクトを作成
ticker = yf.Ticker(ticker_code)

# 1. 企業情報の取得
print("--- 1. 企業情報・指標 ---")
info = ticker.info
print(f"企業名: {info.get('longName')}")
print(f"業種: {info.get('industry')}")
print(f"株価: {info.get('currentPrice', 'N/A'):,.2f} 円")
print(f"PER (株価収益率): {info.get('trailingPE', 'N/A'):.2f} 倍")
print(f"PBR (株価純資産倍率): {info.get('priceToBook', 'N/A'):.2f} 倍")

# 配当利回りはパーセント表示に修正
div_yield = info.get('dividendYield')
if div_yield:
    print(f"配当利回り: {div_yield:.2f}%")
else:
    print(f"配当利回り: N/A")

print("\n" + "="*50 + "\n")


# 2. 株価データとRCIの取得・計算
print(f"--- 2. 最新の株価とRCI (対象期間: {period_to_get}) ---")
hist = ticker.history(period=period_to_get)

# 短期・中期・長期のRCIを計算してDataFrameに追加
hist = add_rci(hist, 9)
hist = add_rci(hist, 26)
hist = add_rci(hist, 52)

# 最新5日間の終値とRCIを表示
print(hist[['Close', 'RCI_9', 'RCI_26', 'RCI_52']].tail())

print("\n" + "="*50 + "\n")


# 3. フィボナッチ・リトレースメントの計算と表示
print(f"--- 3. フィボナッチライン (対象期間: {period_to_get}) ---")
fib_levels = calculate_fibonacci(hist)

# 計算結果を整形して表示
for level_name, level_price in fib_levels.items():
    print(f"  {level_name.rjust(6)}: {level_price:,.2f} 円")

print("\n" + "="*50 + "\n")


# 4. 配当性向の取得
print("--- 4. 配当性向 ---")
payout_ratio = info.get('payoutRatio')
if payout_ratio:
    # 0~1の範囲で返ってくるのでパーセント表示
    print(f"配当性向: {payout_ratio:.2%}")
else:
    print("配当性向データがありません。")

--- 1. 企業情報・指標 ---
企業名: CyberAgent, Inc.
業種: Advertising Agencies
株価: 1,509.50 円
PER (株価収益率): 35.49 倍
PBR (株価純資産倍率): 4.59 倍
配当利回り: 2.19%


--- 2. 最新の株価とRCI (対象期間: 1y) ---
                            Close      RCI_9     RCI_26     RCI_52
Date                                                              
2025-06-11 00:00:00+09:00  1500.0  33.333333  91.452991  90.233074
2025-06-12 00:00:00+09:00  1485.5 -13.333333  90.564103  94.262785
2025-06-13 00:00:00+09:00  1465.5 -73.333333  85.641026  95.031162
2025-06-16 00:00:00+09:00  1508.0 -43.333333  84.888889  95.466576
2025-06-17 00:00:00+09:00  1509.5   8.333333  84.341880  95.910527


--- 3. フィボナッチライン (対象期間: 1y) ---
フィボナッチ・リトレースメント (押し目・サポートの目安)
期間最高値: 1,538.50 円
期間最安値: 785.21 円
------------------------------
  100.0%: 785.21 円
   78.6%: 946.41 円
   61.8%: 1,072.97 円
   50.0%: 1,161.85 円
   38.2%: 1,250.74 円
   23.6%: 1,360.72 円
    0.0%: 1,538.50 円


--- 4. 配当性向 ---
配当性向: 37.66%


In [ ]:
# @title 銘柄コード入力して株価情報を取得する
import yfinance as yf
import pandas as pd
import numpy as np # RCIの計算に必要

# --- 関数定義 ---

def add_rci(df, period):
    """
    DataFrameにRCI（順位相関指数）の列を追加する関数
    
    Args:
        df (pd.DataFrame): 'Close'列を含む株価データ
        period (int): RCIの計算期間
        
    Returns:
        pd.DataFrame: RCI列が追加されたDataFrame
    """
    # RCIを計算するための内部関数
    def rci_calc(series):
        n = len(series)
        if n == 0: # データがない場合の対応
            return np.nan
        # 日付の順位は 1, 2, ..., n
        date_rank = pd.Series(range(1, n + 1))
        # 価格の順位
        price_rank = pd.Series(series).rank(method='first')
        
        # d (順位の差の二乗和) を計算
        # price_rank.reset_index(drop=True) は、rolling適用時にインデックスがずれるのを防ぐため
        d = ((date_rank - price_rank.reset_index(drop=True)) ** 2).sum()
        
        # 分母が0になるのを防ぐ
        denominator = n * (n**2 - 1)
        if denominator == 0:
            return np.nan
            
        # RCIを計算
        return (1 - (6 * d) / denominator) * 100

    column_name = f'RCI_{period}'
    # rolling().apply() を使って各期間でRCIを計算
    # min_periodsを設定することで、データが足りない期間はNaNになるようにする
    df[column_name] = df['Close'].rolling(window=period, min_periods=period).apply(rci_calc, raw=True)
    return df

def calculate_fibonacci(df):
    """
    フィボナッチ・リトレースメントのレベルを計算する関数
    
    Args:
        df (pd.DataFrame): 'High'と'Low'列を含む株価データ
        
    Returns:
        tuple: (フィボナッチレベルの辞書, 期間最高値, 期間最安値)
    """
    if df.empty:
        print("株価データがありません。フィボナッチ・リトレースメントは計算できません。")
        return {}, 0, 0

    # 期間内の最高値と最安値を取得
    high_price = df['High'].max()
    low_price = df['Low'].min()
    
    # 値幅
    diff = high_price - low_price
    
    # フィボナッチ比率
    ratios = [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
    
    levels = {}
    print(f"フィボナッチ・リトレースメント (押し目・サポートの目安)")
    print(f"期間最高値: {high_price:,.2f} 円")
    print(f"期間最安値: {low_price:,.2f} 円")
    print("-" * 30)

    # 各レベルの価格を計算（高値からの下落率で計算）
    for ratio in sorted(ratios, reverse=True):
        level_price = high_price - diff * ratio
        levels[f'{ratio*100:.1f}%'] = level_price
        
    return levels

# --- メイン処理 ---

# Pandasの表示オプションを設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# 銘柄コードの入力を求める
# 日本株の場合は「xxxx.T」の形式で入力するよう促す
ticker_code = input("調べたい日本株の銘柄コードを入力してください (例: 7203.T): ").upper()

# データ取得期間
period_to_get = "1y"   # 過去1年分

# Tickerオブジェクトを作成
ticker = yf.Ticker(ticker_code)

# 企業情報を取得し、取得できない場合は処理を中断
try:
    info = ticker.info
    # infoオブジェクトが空っぽの場合（無効な銘柄コードなど）
    if not info or 'longName' not in info:
        print(f"エラー: 銘柄コード '{ticker_code}' の企業情報が見つかりませんでした。正しいコードか確認してください。")
        exit() # プログラムを終了
except Exception as e:
    print(f"エラー: 銘柄コード '{ticker_code}' の情報取得中に問題が発生しました。インターネット接続を確認してください。詳細: {e}")
    exit() # プログラムを終了


print("--- 1. 企業情報・指標 ---")
print(f"企業名: {info.get('longName', 'N/A')}")
print(f"業種: {info.get('industry', 'N/A')}")
# currentPriceも取得できない場合があるため、N/A表示を追加
print(f"株価: {info.get('currentPrice', 'N/A'):,.2f} 円" if info.get('currentPrice') else "株価: N/A")
print(f"PER (株価収益率): {info.get('trailingPE', 'N/A'):.2f} 倍" if info.get('trailingPE') else "PER: N/A")
print(f"PBR (株価純資産倍率): {info.get('priceToBook', 'N/A'):.2f} 倍" if info.get('priceToBook') else "PBR: N/A")

# 配当利回りはパーセント表示に修正 (yfinanceが返す値がすでにパーセント値であると仮定)
div_yield = info.get('dividendYield')
if div_yield is not None: # Noneチェックを強化
    # yfinanceのdividendYieldはすでにパーセンテージで与えられている場合があるため、
    # そのまま小数としてフォーマットし、文字列として'%'を追加する。
    print(f"配当利回り: {div_yield:.2f}%")
else:
    print(f"配当利回り: N/A")

print("\n" + "="*50 + "\n")


# 2. 株価データとRCIの取得・計算
print(f"--- 2. 最新の株価とRCI (対象期間: {period_to_get}) ---")
hist = ticker.history(period=period_to_get)

if hist.empty:
    print(f"銘柄コード '{ticker_code}' の株価データが取得できませんでした。期間やコードを確認してください。")
else:
    # 短期・中期・長期のRCIを計算してDataFrameに追加
    hist = add_rci(hist, 9)
    hist = add_rci(hist, 26)
    hist = add_rci(hist, 52)

    # 最新5日間の終値とRCIを表示
    print(hist[['Close', 'RCI_9', 'RCI_26', 'RCI_52']].tail())

print("\n" + "="*50 + "\n")


# 3. フィボナッチ・リトレースメントの計算と表示
print(f"--- 3. フィボナッチライン (対象期間: {period_to_get}) ---")
fib_levels = calculate_fibonacci(hist)

# 計算結果を整形して表示
if fib_levels: # fib_levelsが空でないことを確認
    for level_name, level_price in fib_levels.items():
        print(f"  {level_name.rjust(6)}: {level_price:,.2f} 円")

print("\n" + "="*50 + "\n")


# 4. 配当性向の取得
print("--- 4. 配当性向 ---")
payout_ratio = info.get('payoutRatio')
if payout_ratio is not None: # Noneチェックを強化
    # 0~1の範囲で返ってくるのでパーセント表示 (一般的にyfinanceは小数で返す)
    print(f"配当性向: {payout_ratio:.2%}")
else:
    print("配当性向データがありません。")

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime

# --- 関数定義 ---

def add_rci(df, period):
    """
    DataFrameにRCI（順位相関指数）の列を追加する関数
    
    Args:
        df (pd.DataFrame): 'Close'列を含む株価データ
        period (int): RCIの計算期間
        
    Returns:
        pd.DataFrame: RCI列が追加されたDataFrame
    """
    # RCIを計算するための内部関数
    def rci_calc(series):
        n = len(series)
        if n < period: # 計算に必要な期間データが不足している場合
            return np.nan
        if n == 0: # データがない場合の対応
            return np.nan
        
        # 日付の順位は 1, 2, ..., n
        date_rank = pd.Series(range(1, n + 1))
        # 価格の順位
        # rank()のデフォルトは'average'ですが、順位相関指数では'first'が良いとされます
        price_rank = pd.Series(series).rank(method='first')
        
        # d (順位の差の二乗和) を計算
        d = ((date_rank - price_rank.reset_index(drop=True)) ** 2).sum()
        
        # 分母が0になるのを防ぐ
        denominator = n * (n**2 - 1)
        if denominator == 0:
            return np.nan # 計算不能な場合はNaNを返す
            
        # RCIを計算
        return (1 - (6 * d) / denominator) * 100

    column_name = f'RCI_{period}'
    # rolling().apply() を使って各期間でRCIを計算
    # min_periodsを設定することで、データが足りない期間はNaNになるようにする
    # df['Close']が数値型でない場合に備えて数値型に変換
    if not pd.api.types.is_numeric_dtype(df['Close']):
        df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
        
    # 計算前にNaNがないか確認し、NaNがあればfillnaで対処するか、rollingの計算から除外する
    # RCI計算ではNaNを含むデータは正しく計算できないため、na_action='ignore'でスキップ
    df[column_name] = df['Close'].rolling(window=period, min_periods=period, closed='right').apply(rci_calc, raw=True)
    return df

def calculate_fibonacci(df):
    """
    フィボナッチ・リトレースメントのレベルを計算する関数
    
    Args:
        df (pd.DataFrame): 'High'と'Low'列を含む株価データ
        
    Returns:
        dict: フィボナッチレベルの辞書
    """
    # 必要な列が存在するか、データが空でないかを確認
    if df.empty or 'High' not in df.columns or 'Low' not in df.columns:
        print("株価データが不足しているか、必要な列 ('High'/'Low') がありません。フィボナッチ・リトレースメントは計算できません。")
        return {}

    # 期間内の最高値と最安値を取得
    # NaNが含まれている可能性があるため、skipna=Trueで計算
    high_price = df['High'].max(skipna=True)
    low_price = df['Low'].min(skipna=True)
    
    # 最高値または最安値がNaNの場合（データがすべてNaNなど）
    if pd.isna(high_price) or pd.isna(low_price):
        print("最高値または最安値が特定できませんでした。フィボナッチ・リトレースメントは計算できません。")
        return {}

    # 値幅が0の場合（株価が変動していないなど）
    diff = high_price - low_price
    if diff == 0:
        print("株価の変動がないため、フィボナッチ・リトレースメントは計算できません。")
        return {}
    
    # フィボナッチ比率
    ratios = [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
    
    levels = {}
    print(f"フィボナッチ・リトレースメント (押し目・サポートの目安)")
    print(f"期間最高値: {high_price:,.2f} 円")
    print(f"期間最安値: {low_price:,.2f} 円")
    print("-" * 30)

    # 各レベルの価格を計算（高値からの下落率で計算）
    for ratio in sorted(ratios, reverse=True):
        level_price = high_price - diff * ratio
        levels[f'{ratio*100:.1f}%'] = level_price
        
    return levels

# --- メイン処理 ---

# Pandasの表示オプションを設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# 銘柄コードの入力を求める
# 日本株の場合は「xxxx.T」の形式で入力するよう促す
ticker_code = input("調べたい日本株の銘柄コードを入力してください (例: 7203.T): ").upper()

# データ取得期間
period_to_get = "1y"   # 過去1年分

# Tickerオブジェクトを作成
ticker = yf.Ticker(ticker_code)

# 1. 企業情報の取得とエラーチェック
print("--- 1. 企業情報・指標 ---")
try:
    info = ticker.info
    # infoオブジェクトが空っぽの場合（無効な銘柄コードなど）
    if not info or not info.get('longName'):
        print(f"エラー: 銘柄コード '{ticker_code}' の企業情報が見つかりませんでした。正しいコードか確認してください。")
        exit(1) # プログラムを終了
except Exception as e:
    print(f"エラー: 銘柄コード '{ticker_code}' の情報取得中に問題が発生しました。インターネット接続を確認してください。詳細: {e}")
    exit(1) # プログラムを終了


print(f"企業名: {info.get('longName', 'N/A')}")
print(f"業種: {info.get('industry', 'N/A')}")
# currentPriceも取得できない場合があるため、N/A表示を追加
print(f"株価: {info.get('currentPrice', 'N/A'):,.2f} 円" if info.get('currentPrice') else "株価: N/A")
print(f"PER (株価収益率): {info.get('trailingPE', 'N/A'):.2f} 倍" if info.get('trailingPE') else "PER: N/A")
print(f"PBR (株価純資産倍率): {info.get('priceToBook', 'N/A'):.2f} 倍" if info.get('priceToBook') else "PBR: N/A")

# 配当利回りはパーセント表示に修正 (yfinanceが返す値がすでにパーセント値であると仮定)
div_yield = info.get('dividendYield')
if div_yield is not None:
    print(f"配当利回り: {div_yield:.2f}%")
else:
    print(f"配当利回り: N/A")

print("\n" + "="*50 + "\n")


# 2. 株価データとRCIの取得・計算とエラーチェック
print(f"--- 2. 最新の株価とRCI (対象期間: {period_to_get}) ---")
try:
    hist = ticker.history(period=period_to_get)
except Exception as e:
    print(f"エラー: 銘柄コード '{ticker_code}' の株価データ取得中に問題が発生しました。詳細: {e}")
    exit(1)

if hist.empty:
    print(f"銘柄コード '{ticker_code}' の株価データが取得できませんでした。期間やコードを確認してください。")
else:
    # 短期・中期・長期のRCIを計算してDataFrameに追加
    hist = add_rci(hist, 9)
    hist = add_rci(hist, 26)
    hist = add_rci(hist, 52)

    # 最新5日間の終値とRCIを表示
    print(hist[['Close', 'RCI_9', 'RCI_26', 'RCI_52']].tail())

print("\n" + "="*50 + "\n")


# 3. フィボナッチ・リトレースメントの計算と表示
print(f"--- 3. フィボナッチライン (対象期間: {period_to_get}) ---")
# histが空でない場合のみ計算を試みる
if not hist.empty:
    fib_levels = calculate_fibonacci(hist)

    # 計算結果を整形して表示
    if fib_levels: # fib_levelsが空でないことを確認
        for level_name, level_price in fib_levels.items():
            print(f"  {level_name.rjust(6)}: {level_price:,.2f} 円")
else:
    print("株価データがないため、フィボナッチラインは計算できませんでした。")

print("\n" + "="*50 + "\n")


# 4. 配当性向の取得
print("--- 4. 配当性向 ---")
payout_ratio = info.get('payoutRatio')
if payout_ratio is not None:
    print(f"配当性向: {payout_ratio:.2%}")
else:
    print("配当性向データがありません。")

print("\n--- 処理完了 ---")